In [55]:
import xml.etree.ElementTree as ET
import re
import numpy as np
import json
import pandas as pd

In [2]:
tree = ET.parse('data/Tags.xml')
root = tree.getroot()

In [3]:
## abstract tags that appears more than 100 times

In [4]:
index = 0
tagIndexDic = {}
for child in root:
    if int(child.attrib.get("Count")) > 100:
        tagIndexDic[child.attrib.get("TagName")] = index
        index += 1

In [5]:
tagNum = len(tagIndexDic.keys())

In [6]:
## for each post,
process = 0 
# init a matrix that helps counting for each year.
years = [str(year) for year in range(2008, 2021)]
countMat = {}
for year in years:
    countMat[year] = np.zeros((tagNum, tagNum))
    
for event, child in ET.iterparse("data/Posts.xml"):
    if event == "end":
        attr = child.attrib
        if "Tags" in attr:
            # parse time
            if "CreationDate" in attr:
                year = attr["CreationDate"][:4]
                # parse tag
                tags = re.findall(r"<(.+?)>", attr["Tags"])
                for tag1 in tags:
                    for tag2 in tags:
                        if ((tag1 in tagIndexDic) and (tag2 in tagIndexDic)):
                            # count the number into the matrix
                            countMat[year][tagIndexDic.get(tag1)][tagIndexDic.get(tag2)] += 1
    child.clear()
    if (process % 1000000 == 0):
        print(process, " posts have been processed")
    process += 1

0  posts have been processed


1000000  posts have been processed


2000000  posts have been processed


3000000  posts have been processed


4000000  posts have been processed


...

46000000  posts have been processed


47000000  posts have been processed


48000000  posts have been processed


49000000  posts have been processed


50000000  posts have been processed


In [45]:
# compute the top 70 tags each year and their combination and write the data as json files
tagIndexDicRev = {v: k for k, v in tagIndexDic.items()}
for year in years:
    count = np.diag(countMat[year])
    topIndex = np.argsort(count)
    topTag = topIndex[-70:]
    subMatrix = countMat[year][topTag,:][:,topTag]

    nodes = []
    links = []
    for i in range(70):
        nodes.append({"tagName": tagIndexDicRev[topTag[i]], "count":count[topTag[i]]})
        for j in range(i + 1, 70):
            if (subMatrix[i][j] > 0):
                links.append({"source": tagIndexDicRev[topTag[i]], "target": tagIndexDicRev[topTag[j]], "value": subMatrix[i][j]})

    outputJson = {"nodes": nodes, "links": links}
    fileName = "data/com" + year + ".json"
    outputStr = json.dumps(outputJson)
    print("ready to write in file " + year)
    with open(fileName, "w") as f:
        f.write(outputStr)
    print("finish writing the file " + year)

ready to write in file 2008
finish writing the file 2008
ready to write in file 2009
finish writing the file 2009
ready to write in file 2010
finish writing the file 2010
ready to write in file 2011
finish writing the file 2011
ready to write in file 2012
finish writing the file 2012
ready to write in file 2013
finish writing the file 2013
ready to write in file 2014
finish writing the file 2014
ready to write in file 2015
finish writing the file 2015
ready to write in file 2016
finish writing the file 2016
ready to write in file 2017
finish writing the file 2017
ready to write in file 2018
finish writing the file 2018
ready to write in file 2019
finish writing the file 2019
ready to write in file 2020
finish writing the file 2020


In [48]:
# compute the top 200 tags each year and write the data as json files
for year in years:
    data = []
    count = np.diag(countMat[year])
    topIndex = np.argsort(count)
    topTag = topIndex[-200:]
    subMatrix = countMat[year][topTag,:][:,topTag]

    for i in range(200):
        data.append({"tagName": tagIndexDicRev[topTag[i]], "count":count[topTag[i]]})

    fileName = "data/Top200Tag" + year + ".json"
    outputStr = json.dumps(data)
    print("ready to write in file" + year)
    with open(fileName, "w") as f:
        f.write(outputStr)
    print("finish writing the file")

ready to write in file2008
finish writing the file
ready to write in file2009
finish writing the file
ready to write in file2010
finish writing the file
ready to write in file2011
finish writing the file
ready to write in file2012
finish writing the file
ready to write in file2013
finish writing the file
ready to write in file2014
finish writing the file
ready to write in file2015
finish writing the file
ready to write in file2016
finish writing the file
ready to write in file2017
finish writing the file
ready to write in file2018
finish writing the file
ready to write in file2019
finish writing the file
ready to write in file2020
finish writing the file


In [88]:
index = 0
tagIndexDic = {}
for child in root:
    if int(child.attrib.get("Count")) > 1000:
        tagIndexDic[child.attrib.get("TagName")] = index
        index += 1

colNum = len(tagIndexDic.items())
tagIndexDicRev = {v: k for k, v in tagIndexDic.items()}

In [77]:
# generate timeline 2018-01 -- 2020-12
timeline = []
for year in range(2008, 2021):
    for month in range(1, 13):
        timeline.append("{0:d}-{1:02d}".format(year, month))

rowNum = len(timeline)
timeIndexDic = dict(zip(timeline, range(rowNum)))

In [79]:
# count the process,
process = 0 
# init a matrix that helps counting for the tags frequency.
countMat = np.zeros((rowNum, colNum))

for event, child in ET.iterparse("data/Posts.xml"):
    if event == "end":
        attr = child.attrib
        if "Tags" in attr:
            # parse time
            if "CreationDate" in attr:
                time = attr["CreationDate"][:7]
                # parse tag
                tags = re.findall(r"<(.+?)>", attr["Tags"])
                for tag in tags:
                    if (tag in tagIndexDic):
                        # count the number into the matrix
                        countMat[timeIndexDic.get(time)][tagIndexDic.get(tag)] += 1
    child.clear()
    if (process % 1000000 == 0):
        print(process, " posts have been processed")
    process += 1

0  posts have been processed


1000000  posts have been processed


2000000  posts have been processed


3000000  posts have been processed


4000000  posts have been processed


...

46000000  posts have been processed


47000000  posts have been processed


48000000  posts have been processed


49000000  posts have been processed


50000000  posts have been processed


In [87]:
# find the month that does not have data
np.sum(countMat, axis = 1)

array([0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00,
       0.00000e+00, 1.50000e+01, 1.00720e+04, 3.66910e+04, 3.85190e+04,
       3.32710e+04, 3.13550e+04, 4.19180e+04, 4.64820e+04, 5.35770e+04,
       5.60410e+04, 6.80920e+04, 7.40110e+04, 8.48860e+04, 8.50820e+04,
       8.60860e+04, 9.48250e+04, 9.94020e+04, 9.73480e+04, 1.16549e+05,
       1.17994e+05, 1.36103e+05, 1.30992e+05, 1.35912e+05, 1.44225e+05,
       1.56792e+05, 1.63717e+05, 1.58389e+05, 1.65110e+05, 1.80889e+05,
       1.81050e+05, 2.11663e+05, 2.18830e+05, 2.66264e+05, 2.53346e+05,
       2.67810e+05, 2.67366e+05, 2.72826e+05, 2.86724e+05, 2.72113e+05,
       2.72163e+05, 2.93517e+05, 2.79201e+05, 3.15273e+05, 3.33170e+05,
       3.59109e+05, 3.48418e+05, 3.63170e+05, 3.52985e+05, 3.82938e+05,
       3.81529e+05, 3.57678e+05, 4.03005e+05, 3.98495e+05, 3.64828e+05,
       4.25290e+05, 4.12292e+05, 4.69113e+05, 4.65480e+05, 4.50084e+05,
       4.28311e+05, 4.77471e+05, 4.61832e+05, 4.57268e+05, 5.097

In [94]:
df = pd.DataFrame(countMat)
df.columns = [tagIndexDicRev[i] for i in range(colNum)]
df.index = timeline
df = df.drop(timeline[:6])
df = df.drop(timeline[-3:])
df

,.net,html,javascript,css,php,c,c#,c++,ruby,lisp,...,ios13,swiftui,xcode11,macos-catalina,confluent-platform,laravel-6,angular9,asp.net-core-3.1,laravel-7,microsoft-graph-api
2008-07,1.0,1.0,0.0,1.0,0.0,0.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2008-08,442.0,112.0,162.0,62.0,161.0,85.0,511.0,164.0,73.0,9.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2008-09,1542.0,327.0,640.0,237.0,482.0,320.0,1649.0,755.0,290.0,41.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2008-10,1566.0,332.0,725.0,225.0,617.0,303.0,1989.0,811.0,249.0,22.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2008-11,1203.0,329.0,580.0,203.0,504.0,259.0,1730.0,735.0,160.0,19.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-05,1526.0,10062.0,25175.0,6534.0,7907.0,3600.0,9915.0,7059.0,774.0,41.0,...,83.0,939.0,73.0,128.0,48.0,87.0,307.0,164.0,230.0,274.0
2020-06,1471.0,9295.0,23347.0,6055.0,7175.0,2751.0,9041.0,6147.0,670.0,20.0,...,66.0,934.0,51.0,101.0,81.0,86.0,312.0,155.0,238.0,299.0
2020-07,1462.0,9026.0,23440.0,5925.0,7003.0,2259.0,8946.0,5647.0,760.0,21.0,...,58.0,1070.0,51.0,86.0,68.0,77.0,219.0,213.0,258.0,282.0
2020-08,1385.0,8603.0,22440.0,5776.0,6878.0,2163.0,8885.0,5493.0,633.0,13.0,...,55.0,951.0,41.0,95.0,42.0,48.0,160.0,172.0,255.0,282.0


In [97]:
df.to_csv("data/monthlyCountPerTag.csv", index=False)

In [98]:
# count the process,
process = 0 
# init a matrix that helps counting for the tags frequency.
countMat = np.zeros((rowNum, 1))

for event, child in ET.iterparse("data/Posts.xml"):
    if event == "end":
        attr = child.attrib
        if "CreationDate" in attr:
            time = attr["CreationDate"][:7]
            countMat[timeIndexDic.get(time)][0] += 1
    child.clear()
    if (process % 1000000 == 0):
        print(process, " posts have been processed")
    process += 1

0  posts have been processed


1000000  posts have been processed


2000000  posts have been processed


3000000  posts have been processed


4000000  posts have been processed


...

46000000  posts have been processed


47000000  posts have been processed


48000000  posts have been processed


49000000  posts have been processed


50000000  posts have been processed


In [99]:
totalPostCount = []
for i in range(6, rowNum - 3):
    totalPostCount.append({"date": timeline[i], "count": countMat[i][0]})

outputData = json.dumps(totalPostCount)
fileName = "data/totalPostCountPerMonth.json"
with open(fileName, "w") as f:
    f.write(outputData)
print(countMat)

[[0.00000e+00]
 [0.00000e+00]
 [0.00000e+00]
 [0.00000e+00]
 [0.00000e+00]
 [0.00000e+00]
 [6.00000e+00]
 [1.90640e+04]
 [7.57810e+04]
 [6.92790e+04]
 [5.39260e+04]
 [5.27010e+04]
 [6.94450e+04]
 [7.58050e+04]
 [8.28780e+04]
 [8.50040e+04]
 [1.00534e+05]
 [1.09283e+05]
 [1.21420e+05]
 [1.18105e+05]
 [1.16499e+05]
 [1.27898e+05]
 [1.38024e+05]
 [1.35436e+05]
 [1.54675e+05]
 [1.48766e+05]
 [1.68961e+05]
 [1.58054e+05]
 [1.63189e+05]
 [1.72807e+05]
 [1.89244e+05]
 [1.95822e+05]
 [1.85975e+05]
 [1.92825e+05]
 [2.13742e+05]
 [2.11107e+05]
 [2.42742e+05]
 [2.48620e+05]
 [3.04816e+05]
 [2.85360e+05]
 [2.96127e+05]
 [2.93871e+05]
 [2.96034e+05]
 [3.14693e+05]
 [2.93473e+05]
 [2.88760e+05]
 [3.14124e+05]
 [2.99384e+05]
 [3.34419e+05]
 [3.52057e+05]
 [3.77693e+05]
 [3.59400e+05]
 [3.73808e+05]
 [3.61180e+05]
 [3.90725e+05]
 [3.90860e+05]
 [3.61569e+05]
 [4.11747e+05]
 [3.99370e+05]
 [3.68247e+05]
 [4.27441e+05]
 [4.12057e+05]
 [4.67360e+05]
 [4.57559e+05]
 [4.44582e+05]
 [4.19545e+05]
 [4.68850e

In [110]:
topTagSet = set()
for year in years:
    filename = "data/top200Tag" + year + ".json"
    with open(filename) as f:
        data = json.load(f)

    tags = [data[i]["tagName"] for i in range((len(data)))]
    topTagSet.update(tags)

In [112]:
len(topTagSet)

445

In [113]:
topTagList = list(topTagSet)
df = df[topTagList]

,search,tkinter,security,matplotlib,visual-c++,ios5,sql,events,google-cloud-functions,asp.net-mvc-2,...,memory,design-patterns,web-applications,curl,web-scraping,image-processing,zend-framework,file,windows-xp,servlets
2008-07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2008-08,15.0,1.0,55.0,0.0,15.0,0.0,147.0,18.0,0.0,0.0,...,25.0,29.0,28.0,4.0,1.0,3.0,2.0,24.0,10.0,3.0
2008-09,47.0,9.0,166.0,2.0,59.0,0.0,503.0,52.0,0.0,0.0,...,57.0,110.0,89.0,6.0,0.0,12.0,11.0,65.0,45.0,29.0
2008-10,50.0,4.0,165.0,0.0,70.0,0.0,534.0,84.0,0.0,0.0,...,59.0,108.0,64.0,11.0,2.0,26.0,10.0,101.0,33.0,27.0
2008-11,44.0,2.0,128.0,0.0,53.0,0.0,414.0,59.0,0.0,0.0,...,35.0,78.0,65.0,10.0,1.0,24.0,15.0,81.0,27.0,27.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-05,291.0,859.0,393.0,1396.0,165.0,0.0,5280.0,244.0,497.0,2.0,...,279.0,194.0,167.0,394.0,1001.0,473.0,11.0,800.0,6.0,114.0
2020-06,236.0,783.0,379.0,1236.0,185.0,0.0,4946.0,178.0,408.0,0.0,...,268.0,137.0,167.0,371.0,923.0,406.0,16.0,606.0,8.0,101.0
2020-07,259.0,930.0,390.0,1186.0,172.0,0.0,4671.0,219.0,391.0,0.0,...,243.0,157.0,175.0,302.0,865.0,355.0,12.0,610.0,3.0,114.0
2020-08,230.0,949.0,381.0,950.0,120.0,1.0,4364.0,185.0,384.0,0.0,...,252.0,169.0,169.0,336.0,819.0,310.0,20.0,573.0,1.0,117.0


In [114]:
df.to_csv("data/monthlyCountPerTag.csv", index=True)